In [96]:
import pandas as pd
import numpy as np
import re
import warnings

warnings.filterwarnings("ignore")

year_List = list(range(2016,2022))
year_List = [value for value in year_List if value != 2017]   # 2017년도 빼기

category = ["연령별", "가구소득별"]

In [130]:
def leisure_pretreat(dframe) : 

    """Ver. 1
    
    여가 관련 통계조사 전처리 함수입니다.
    
    공공데이터의 csv 형식이 같을 것이라 생각했습니다.

    """

    year_List = list(range(2016,2022))
    year_List = [value for value in year_List if value != 2017]   # 2017년도 빼기

    category = ["연령별", "가구소득별"]

    leis_sep = dframe.iloc[:,:2]

    conc_list_age = []
    conc_list_inc = []

    for yr in year_List : 

        pattern = str(yr)
        df_name = "DataFrame_" + str(yr)

        globals()[df_name] = leisure2.filter(like=pattern)    # 조건에 만족하는 데이터 프레임 반환
        globals()[df_name] = pd.concat([leis_sep, globals()[df_name]], axis=1)     # 왼쪽 오른쪽으로 합치기, 인덱스가 다르면 outer join
        globals()[df_name] = globals()[df_name].replace("-",0)
        globals()[df_name].columns = globals()[df_name].iloc[0,:].to_list()
        globals()[df_name] = globals()[df_name].iloc[1:,:]
        globals()[df_name].iloc[:,2:] = globals()[df_name].iloc[:,2:].astype(float)
        globals()[df_name].iloc[:,3:] = globals()[df_name].iloc[:,3:].apply(lambda x : x * globals()[df_name].iloc[:,2] / 100)  # 백분율로 표시된 값을 실제 값으로 변환
        globals()[df_name] = globals()[df_name].drop("표본수",axis=1)  # 합계 부분 삭제
        globals()[df_name].iloc[:,2:] = globals()[df_name].iloc[:,2:].astype(int)

        for cat1 in category : 

            df_name_c1 = "DataFrame_" + str(yr) + cat1
            globals()[df_name_c1] = globals()[df_name][globals()[df_name]["통계분류(1)"]==cat1].iloc[:,1:]
            globals()[df_name_c1].insert(0, "연도별", yr)
            globals()[df_name_c1].rename({"통계분류(2)":cat1},axis=1,inplace=True)

            if cat1 == category[0] : 
                conc_list_age.append(globals()[df_name_c1])
            else : 
                conc_list_inc.append(globals()[df_name_c1])

    df1 = pd.concat(conc_list_age, axis=0)
    df2 = pd.concat(conc_list_inc, axis=0)

    return df1, df2

# 여가 관련 자료

## 일과 여가 생활 균형

In [ ]:
leisure = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')
# leisure.info()
# leisure.head(10)

In [ ]:
leisure_T = leisure.T   # 행열 전환
# leisure_T.head(5)

In [ ]:
leisure_T.replace("-",0,inplace=True)

In [ ]:
leis_uq = leisure.iloc[:,0].unique()
leis_uq = leis_uq.tolist()   # ndarray -> list
leis_uq = [re.sub(r"\(+.\)","",uq.replace(" ","_")) for uq in leis_uq]  # 괄호와 괄호 안 글자, 공백 제거
print(leis_uq)

In [ ]:
leisure_T.columns = leisure_T.iloc[0,:]
# leisure_T.info()

In [ ]:
columns_nospace = [re.sub(r"\(+.\)", "", col.replace(" ","_")) for col in leisure_T.columns]

In [ ]:
leisure_T.columns = columns_nospace
print(leisure_T.columns)

In [ ]:
# 통계 분류별로 데이터 프레임 분류

for cat in leis_uq : 
    globals()["leisure_{}".format(cat)] = leisure_T[cat]
    
    if type(globals()["leisure_{}".format(cat)]) == pd.core.series.Series :   # type이 시리즈라면
        globals()["leisure_{}".format(cat)] = pd.DataFrame(globals()["leisure_{}".format(cat)])   # 시리즈를 데이터프레임으로 변경
        
    globals()["leisure_{}".format(cat)].columns = globals()["leisure_{}".format(cat)].iloc[1,:]
    globals()["leisure_{}".format(cat)] = globals()["leisure_{}".format(cat)].iloc[2:,:]
    globals()["leisure_{}".format(cat)].iloc[:,1:] = globals()["leisure_{}".format(cat)].iloc[:,1:].astype(float)

In [ ]:
leis_uq2 = leis_uq[1:]
print(leis_uq2)

In [ ]:
for uq2 in leis_uq2 : 
    globals()["leis_{0}_{1}".format(uq2, "합본")] = pd.concat([leisure_통계분류, globals()["leisure_{}".format(uq2)]],axis=1)

In [ ]:
for uniq in leis_uq2 : 
    idx_List = globals()["leis_{0}_{1}".format(uniq, "합본")].index.to_list()   # 분리된 데이터프레임 인덱스 리스트로 반환
    
    for yr in year_List : 
        sep_year = []    # 정규표현식에 만족하는 인덱스를 저장할 리스트
        
        for idx in idx_List :
            find_yr = re.findall(str(yr)+"[.0-9]*",idx)     # 연도 + '.' or 숫자 1개 이상 포함돼 있는 인덱스 저장 

            if find_yr:      # 빈 리스트가 아니라면 (findall은 조건에 안 맞으면 빈 리스트 반환)
                sep_year.append(find_yr[0])

        Last_idx = len(sep_year)
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, "합본")][:][sep_year[0]:sep_year[Last_idx-1]]    # 행 인덱스로 슬라이싱, 행 인덱스로 슬라이싱할 때 콜론 뒤 값을 포함해서 슬라이싱한다.
        pattern = str(yr) + r'[.0-9]*'
        new_index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.map(lambda x: re.sub(pattern, str(yr), x))   # map 함수를 이용해 인덱스마다 정규 표현식 sub를 적용
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = new_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.astype(int)   # 인덱스를 int 타입으로
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"통계분류(2)" : "분류"}, inplace=True)   # 컬럼명 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].reset_index()   # reset_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"index" : "연도"}, inplace=True)    # 인덱스가 컬럼으로 바뀌고, 그 컬럼명을 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].set_index(["연도","분류"])   # 멀티 인덱스 설정


# 전체 -> 리스트 중에서 2016년이면 리스트에 저장 -> 리스트에 저장된 첫번째와 마지막 인덱스로 슬라이싱 -> 새로운 데이터 프레임 생성

In [ ]:
leis_17개_시도별_2016_T = leis_17개_시도별_2016.T
leis_17개_시도별_2016_T.head(10)

In [ ]:
df = leis_17개_시도별_2016_T.stack(level=0)
pd.DataFrame(df)


In [ ]:
pd.concat([leis_17개_시도별_2016, leis_가구소득별_2016],axis=1)


## 일과 여가 생활 균형 2

In [ ]:
leisure2 = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')

In [128]:
balance_age.head()

,연도별,연령별,일에 더 집중하고있다,일에 조금 더 집중하고 있다,다소 일에 집중하고 있다,균형을 이루고 있다,다소 여가에 더 집중하고있다,여가에 조금 더 집중하고 있다,여가에 더 집중하고 있다
4,2016,15~19세,15,95,218,284,159,48,4
5,2016,20대,11,116,302,583,320,135,5
6,2016,30대,34,162,392,599,339,175,5
7,2016,40대,45,195,443,678,426,197,1
8,2016,50대,29,183,404,690,422,217,7


In [129]:
A, B = leisure_pretreat(leisure2)
A.head()

,연도별,연령별,일에 더 집중하고있다,일에 조금 더 집중하고 있다,다소 일에 집중하고 있다,균형을 이루고 있다,다소 여가에 더 집중하고있다,여가에 조금 더 집중하고 있다,여가에 더 집중하고 있다
4,2016,15~19세,15,95,218,284,159,48,4
5,2016,20대,11,116,302,583,320,135,5
6,2016,30대,34,162,392,599,339,175,5
7,2016,40대,45,195,443,678,426,197,1
8,2016,50대,29,183,404,690,422,217,7


## 